## Implementing binary decision trees

In [1]:
import graphlab
import numpy as np
import pandas as pd

In [2]:
loans = graphlab.SFrame("E:\\Machine Learning\\U.W\\Classification\\lending-club-data.gl/")
loans.save("E:\\Machine Learning\\U.W\\Classification\\lending-club-data.csv", format="csv")
loans = pd.read_csv("E:\\Machine Learning\\U.W\\Classification\\lending-club-data.csv")

This non-commercial license of GraphLab Create for academic use is assigned to lxn1021@gmail.com and will expire on November 18, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Xiaoning\AppData\Local\Temp\graphlab_server_1558481550.log.0
c:\python27\lib\site-packages\IPython\core\interactiveshell.py:2714: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


In [7]:
loans["safe_loans"] = loans["bad_loans"].apply(lambda x: +1 if x==0 else -1)
loans = loans.drop(["bad_loans"], axis=1)

In [10]:
features = ["grade", "term", "home_ownership", "emp_length"]
target = "safe_loans"

loans = loans[features + [target]]

In [12]:
loans.head()

,grade,term,home_ownership,emp_length,safe_loans
0,B,36 months,RENT,10+ years,1
1,C,60 months,RENT,< 1 year,-1
2,C,36 months,RENT,10+ years,1
3,C,36 months,RENT,10+ years,1
4,A,36 months,RENT,3 years,1


## Subsample dataset to make sure classes are balanced

In [14]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(frac=percentage, random_state=42)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                :", len(safe_loans)/float(len(loans_data))
print "Percentage of risky loans               :", len(risky_loans)/float(len(loans_data))
print "Total number of loans in our new dataset:", len(loans_data)

Percentage of safe loans                : 0.5
Percentage of risky loans               : 0.5
Total number of loans in our new dataset: 46300


## Transform categorical data into binary features

In [34]:
one_hot_grade = pd.get_dummies(loans_data["grade"], prefix="grade")
one_hot_term = pd.get_dummies(loans_data["term"], prefix="term")
one_hot_ownership = pd.get_dummies(loans_data["home_ownership"], prefix="home_ownership")
one_hot_emp = pd.get_dummies(loans_data["emp_length"], prefix="emp_length")

In [42]:
loans_data = pd.concat([one_hot_grade, one_hot_term, one_hot_ownership, one_hot_emp, loans_data["safe_loans"]], axis=1)

In [69]:
features = loans_data.columns.values 
features = features[features != "safe_loans"]
features

array(['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'term_ 36 months', 'term_ 60 months',
       'home_ownership_MORTGAGE', 'home_ownership_OTHER',
       'home_ownership_OWN', 'home_ownership_RENT', 'emp_length_1 year',
       'emp_length_10+ years', 'emp_length_2 years', 'emp_length_3 years',
       'emp_length_4 years', 'emp_length_5 years', 'emp_length_6 years',
       'emp_length_7 years', 'emp_length_8 years', 'emp_length_9 years',
       'emp_length_< 1 year'], dtype=object)

In [70]:
print "Number of features (after binarizing categorical variables) = %s" % len(features)

Number of features (after binarizing categorical variables) = 24


In [73]:
loans_data["grade_A"]

1         0
6         0
7         0
10        0
12        0
18        0
21        0
23        0
24        0
41        1
45        0
48        0
50        0
58        0
60        0
63        0
87        0
89        0
93        0
102       0
108       0
111       0
118       0
124       1
132       0
136       0
138       0
140       0
151       1
158       0
         ..
59302     0
77224     0
18741     0
17317     1
21021     1
29577     0
24427     0
28755     0
59504     0
48055     0
52456     0
91559     0
14479     0
118235    0
115801    0
50842     0
120961    0
5449      0
54905     0
31383     0
10865     1
78186     0
73459     0
73923     1
88033     0
54114     0
72079     1
90112     0
26711     0
39278     1
Name: grade_A, Length: 46300, dtype: uint8

In [74]:
print "Total number of grade_A loans: %s" % loans_data["grade_A"].sum()

Total number of grade_A loans: 6390


## Train-test split

In [75]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(loans_data, test_size=0.2, random_state=42)

In [77]:
print len(train_data)/float(len(loans_data))
print len(test_data)/float(len(loans_data))

0.8
0.2


# Decision tree implementation

## Function to count number of mistakes while predicting majority class

In [78]:
def intermediate_node_num_mistakes(labels_in_node):
    if len(labels_in_node) == 0:
        return 0
    
    safe_loans = (labels_in_node == 1).sum()
    risky_loans = (labels_in_node == -1).sum()
    
    if safe_loans <= risky_loans:
        return safe_loans
    else:
        return risky_loans

In [79]:
# Test case 1
example_labels = graphlab.SArray([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print "Test passed!"
else:
    print "Test 1 failed... try again!"
    
# Test case 2
example_labels = graphlab.SArray([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print "Test passed!"
else:
    print "Test 2 failed... try again!"

# Test case 3
example_labels = graphlab.SArray([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print "Test passed!"
else:
    print "Test 3 failed... try again!"

Test passed!
Test passed!
Test passed!


## Function to pick best feature to split on

$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

In [80]:
def best_splitting_feature(data, features, target):
    
    best_feature = None
    best_error = 10
    
    num_data_points = float(len(data))
    
    for feature in features:
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        
        left_mistakes = intermediate_node_num_mistakes(left_split[target])
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
        
        error = (left_mistakes + right_mistakes) / num_data_points
        
        if error < best_error:
            best_feature = feature
            best_error = error
    
    
    return best_feature

## Building the tree

In [83]:
def create_leaf(target_values):
    
    leaf = {"splitting_feature": None,
           "left": None,
           "right": None,
           "is_leaf": True}
    
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    if num_ones > num_minus_ones:
        leaf["prediction"] = 1
    else:
        leaf["prediction"] = -1
        
        
    return leaf

In [89]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:]
    
    target_values = data[target]
    print "----------------------------------------------------------------------"
    print "Subtree, depth= %s (%s data points)." % (current_depth, len(target_values))
    
    
    if intermediate_node_num_mistakes(target_values) == 0:
        print "Stopping condition 1 reached."
        return create_leaf(target_values)
    
    if remaining_features == []:
        print "Stopping condition 2 reached."
        return create_leaf(target_values)
    
    if current_depth >= max_depth:
        print "Reached maximum depth. Stopping for now."
        return create_leaf(target_values)
    
    splitting_feature = best_splitting_feature(data, remaining_features, target)    
    
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    remaining_features = remaining_features[remaining_features != splitting_feature]
    print "Split on feature %s. (%s, %s)" % (splitting_feature, len(left_split), len(right_split))
    
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "Creating leaf node."
        return create_leaf(right_split[target])
    
    
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth)
    
    return {"is_leaf": False,
           "prediction": None,
           "splitting_feature": splitting_feature,
           "left": left_tree,
           "right": right_tree}

In [90]:
def count_nodes(tree):
    if tree["is_leaf"]:
        return 1
    
    return 1 + count_nodes(tree["left"]) + count_nodes(tree["right"])

In [92]:
small_data_decision_tree = decision_tree_create(train_data, features, "safe_loans", max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print "Test passed!"
else:
    print "Test failed... try again!"
    print "Number of nodes found               :", count_nodes(small_data_decision_tree)
    print "Number of nodes that should be there: 13"

----------------------------------------------------------------------
Subtree, depth= 0 (37040 data points).


c:\python27\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  del sys.path[0]


Split on feature term_ 36 months. (9199, 27841)
----------------------------------------------------------------------
Subtree, depth= 1 (9199 data points).
Split on feature grade_A. (9102, 97)
----------------------------------------------------------------------
Subtree, depth= 2 (9102 data points).
Split on feature grade_B. (8064, 1038)
----------------------------------------------------------------------
Subtree, depth= 3 (8064 data points).
Reached maximum depth. Stopping for now.
----------------------------------------------------------------------
Subtree, depth= 3 (1038 data points).
Reached maximum depth. Stopping for now.
----------------------------------------------------------------------
Subtree, depth= 2 (97 data points).
Split on feature home_ownership_RENT. (75, 22)
----------------------------------------------------------------------
Subtree, depth= 3 (75 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------

## Build the tree!

In [93]:
my_decision_tree = decision_tree_create(train_data, features, target, current_depth = 0, max_depth = 6)

----------------------------------------------------------------------
Subtree, depth= 0 (37040 data points).


c:\python27\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  del sys.path[0]


Split on feature term_ 36 months. (9199, 27841)
----------------------------------------------------------------------
Subtree, depth= 1 (9199 data points).
Split on feature grade_A. (9102, 97)
----------------------------------------------------------------------
Subtree, depth= 2 (9102 data points).
Split on feature grade_B. (8064, 1038)
----------------------------------------------------------------------
Subtree, depth= 3 (8064 data points).
Split on feature grade_C. (5895, 2169)
----------------------------------------------------------------------
Subtree, depth= 4 (5895 data points).
Split on feature grade_D. (3860, 2035)
----------------------------------------------------------------------
Subtree, depth= 5 (3860 data points).
Split on feature home_ownership_OTHER. (3859, 1)
----------------------------------------------------------------------
Subtree, depth= 6 (3859 data points).
Reached maximum depth. Stopping for now.
------------------------------------------------------

## Making predictions with a decision tree

In [94]:
def classify(tree, x, annotate = False):
    if tree["is_leaf"]:
        if annotate:
            print "At leaf, predicting %s" % tree["prediction"]
        return tree["prediction"]
    else:
        split_feature_value = x[tree["splitting_feature"]]
        if annotate:
            print "Split on %s = %s" % (tree["splitting_feature"], split_feature_value)
        if split_feature_value == 0:
            return classify(tree["left"], x, annotate)
        else:
            return classify(tree["right"], x, annotate)

In [97]:
test_data.iloc[0]

grade_A                    0
grade_B                    0
grade_C                    0
grade_D                    1
grade_E                    0
grade_F                    0
grade_G                    0
term_ 36 months            1
term_ 60 months            0
home_ownership_MORTGAGE    0
home_ownership_OTHER       0
home_ownership_OWN         0
home_ownership_RENT        1
emp_length_1 year          0
emp_length_10+ years       0
emp_length_2 years         0
emp_length_3 years         1
emp_length_4 years         0
emp_length_5 years         0
emp_length_6 years         0
emp_length_7 years         0
emp_length_8 years         0
emp_length_9 years         0
emp_length_< 1 year        0
safe_loans                 1
Name: 53330, dtype: int64

In [99]:
print "Predicted class: %s" % classify(my_decision_tree, test_data.iloc[0])

Predicted class: -1


In [100]:
classify(my_decision_tree, test_data.iloc[0], annotate=True)

Split on term_ 36 months = 1
Split on grade_D = 1
At leaf, predicting -1


-1

## Evaluating your decision tree

$$
\mbox{classification error} = \frac{\mbox{# mistakes}}{\mbox{# total examples}}
$$

In [125]:
def evaluate_classification_error(tree, data, target):
    prediction = []
    for i in range(len(test_data)):
        prediction.append(classify(my_decision_tree, data.iloc[i]))
    
    error = (np.array(data[target]) != np.array(prediction)).sum()/float(len(data))
    
    
    return error

In [126]:
evaluate_classification_error(my_decision_tree, test_data, target)

0.3769978401727862